**Importing sys to library to do system activities**

In [1]:
import sys

**Enter the ticker and the date range you want the data for**

In [50]:
ticker = 'HLT'
start_date = '2015-04-01'
end_date = '2021-02-12'

**We need Yahoo Finance to get the stock data. Using sys, we check if it is already in our system, else we shall install it**

In [51]:
!{sys.executable} -m pip install yfinance 
import yfinance as yf 

**Create a ticker_data object to store data from Yahoo Finance using the selected Ticker and Date Range**

In [52]:
ticker_data = yf.Ticker(ticker)

df = ticker_data.history(start = start_date, end = end_date)

print(df)

                  Open        High  ...  Dividends  Stock Splits
Date                                ...                         
2015-04-01   58.316563   58.355862  ...        0.0           0.0
2015-04-02   57.628870   58.375509  ...        0.0           0.0
2015-04-06   57.471678   58.414802  ...        0.0           0.0
2015-04-07   58.375516   58.375516  ...        0.0           0.0
2015-04-08   59.102514   59.986695  ...        0.0           0.0
...                ...         ...  ...        ...           ...
2021-02-05  111.300003  112.599998  ...        0.0           0.0
2021-02-08  112.690002  113.160004  ...        0.0           0.0
2021-02-09  112.239998  112.510002  ...        0.0           0.0
2021-02-10  110.169998  110.489998  ...        0.0           0.0
2021-02-11  110.139999  111.169998  ...        0.0           0.0

[1478 rows x 7 columns]


**Import Pandas and Numpy to do some data manipulations**

In [53]:
import pandas as pd
import numpy as np

**Since the Date in df is not a Pandas Date Type, we convert it using pd.to_datetime**

In [54]:
date_change = '%Y-%m-%d'

df['Date'] = df.index

df['Date'] = pd.to_datetime(df['Date'], format = date_change)

Dates = df['Date']

**Since we have only limited features in our stock data, import the Technical Analysis (ta) package to add more features**

In [55]:
!{sys.executable} -m pip install ta
from ta import add_all_ta_features 

**Add these technical indicators to the dataframe df**

In [56]:
df = add_all_ta_features(df, 'Open', 'High', 'Low', 'Close', 'Volume', fillna = True)
print(df.columns)

/usr/local/lib/python3.6/dist-packages/ta/trend.py:768: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.6/dist-packages/ta/trend.py:772: RuntimeWarning:

invalid value encountered in double_scalars



Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Date', 'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi',
       'volume_mfi', 'volume_em', 'volume_sma_em', 'volume_vpt', 'volume_nvi',
       'volume_vwap', 'volatility_atr', 'volatility_bbm', 'volatility_bbh',
       'volatility_bbl', 'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
       'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm',
       'volatility_dcw', 'volatility_dcp', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow', 'trend_adx',
       'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
       'trend_vortex_ind_neg', 'trend_vortex_ind_diff', 'trend_trix',
       'trend_mass_index', 'trend_cci', 'trend_

**The technical indicators have been added to our dataframe. You can see what each of them mean in the [dictionary](https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html)**

**Now, install and import the FastAI package to extract important information from each date like day of the month, year or quarter**

In [57]:
!{sys.executable} -m pip install fastai 
import fastai.tabular 

**Now, define the date parts and add them to the df**

In [58]:
fastai.tabular.add_datepart(df, 'Date', drop = True)

df['Date'] = pd.to_datetime(df.index.values, format = date_change)

fastai.tabular.add_cyclic_datepart(df, 'Date', drop = True)

/usr/local/lib/python3.6/dist-packages/fastai/tabular/transform.py:63: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,Open,High,Low,Close,Volume,Dividends,Stock Splits,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,...,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,weekday_cos,weekday_sin,day_month_cos,day_month_sin,month_year_cos,month_year_sin,day_year_cos,day_year_sin
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-04-01,58.316563,58.355862,57.157307,57.530628,2307554,0.0,0.0,-8.700577e+05,2307554,-0.377048,0.000000e+00,50.000000,0.000000,0.000000,-433533.976950,1000.000000,57.681266,0.000000,57.530628,57.530628,57.530628,0.000000,0.000000,0.0,0.0,57.681266,58.879821,56.482711,4.155787,0.437159,0.0,0.0,57.157307,58.355862,57.756584,2.083334,0.311476,0.000000,0.000000,0.000000,...,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.147621,31.147621,-68.852379,0.000000,57.530628,0.000000,0.000000,0.000000,0.000000,-18.849473,0.000000,0.000000,2015,4,14,1,2,91,False,True,False,True,False,False,1427846400,-0.222521,0.974928,1.000000e+00,0.000000,6.123234e-17,1.0,0.021516,0.999769
2015-04-02,57.628870,58.375509,57.569923,57.687813,3461306,0.0,0.0,-3.318304e+06,5768860,-0.575210,5.440640e+05,100.000000,5.030253,5.030253,-425504.824290,1000.000000,57.799155,0.000000,57.609221,57.766405,57.452036,0.545693,0.750000,0.0,0.0,57.779507,58.781577,56.777436,3.468602,0.454248,0.0,0.0,57.157307,58.375509,57.766408,2.114596,0.435483,0.000000,0.012539,0.002508,...,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,7.692173,43.548280,37.347950,-56.451720,0.000000,57.582683,0.000000,3.846073,0.769215,3.076859,0.273219,0.272846,0.273219,2015,4,14,2,3,92,False,False,False,False,False,False,1427932800,-0.900969,0.433884,9.781476e-01,0.207912,6.123234e-17,1.0,0.004304,0.999991
2015-04-06,57.471678,58.414802,57.412731,58.198669,3101218,0.0,0.0,-1.554863e+06,8870078,-0.175293,6.926660e+05,100.000000,-1.904780,1.562736,36919.899647,1008.855538,57.872429,0.000000,57.805703,58.376068,57.235339,1.973385,0.844487,0.0,0.0,57.855916,58.857987,56.853845,3.464021,0.671023,0.0,0.0,57.157307,58.414802,57.786054,2.175382,0.828125,0.000000,0.062972,0.014601,...,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,30.967687,82.812473,52.502791,-17.187527,0.000000,57.811097,0.000000,5.494728,1.714317,3.780411,0.885554,0.881656,1.161192,2015,4,15,6,0,96,False,False,False,False,False,False,1428278400,1.000000,0.000000,5.000000e-01,0.866025,6.123234e-17,1.0,-0.064508,0.997917
2015-04-07,58.375516,58.375516,57.314499,57.353798,1719055,0.0,0.0,-3.146573e+06,7151023,-0.297151,3.862307e+05,79.315845,-4.243866,-0.372798,2507.390506,994.209957,57.841396,0.000000,57.692727,58.322926,57.062528,2.184674,0.231094,0.0,0.0,57.812255,58.829062,56.795447,3.517619,0.274561,0.0,0.0,57.157307,58.414802,57.786054,2.179642,0.156256,0.000000,0.034370,0.018555,...,0.000000,41.924049,0.000000,0.000000,0.000000,89.182944,23.923445,15.625599,47.328784,-84.374401,0.000000,57.635893,0.000000,2.331071,1.837668,0.493403,-1.451702,-1.462343,-0.307367,2015,4,15,7,1,97,False,False,False,False,False,False,1428364800,0.623490,0.781831,3.090170e-01,0.951057,6.123234e-17,1.0,-0.081676,0.996659
2015-04-08,59.102514,59.986695,58.591655,59.947399,5736940,0.0,0.0,2.267171e+06,12887963,0.

**Defining the days ahead we want to predict for is Shifts. Also define the training set percentage and the plot dimensions**

In [59]:
shifts = [1, 5, 10]
train_pct = .75
w = 16
h = 14

**Dataframe for reference**

In [60]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,...,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,weekday_cos,weekday_sin,day_month_cos,day_month_sin,month_year_cos,month_year_sin,day_year_cos,day_year_sin
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-04-01,58.316563,58.355862,57.157307,57.530628,2307554,0.0,0.0,-8.700577e+05,2307554,-0.377048,0.000000e+00,50.000000,0.000000,0.000000,-433533.976950,1000.000000,57.681266,0.000000,57.530628,57.530628,57.530628,0.000000,0.000000,0.0,0.0,57.681266,58.879821,56.482711,4.155787,0.437159,0.0,0.0,57.157307,58.355862,57.756584,2.083334,0.311476,0.000000,0.000000,0.000000,...,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.147621,31.147621,-68.852379,0.000000,57.530628,0.000000,0.000000,0.000000,0.000000,-18.849473,0.000000,0.000000,2015,4,14,1,2,91,False,True,False,True,False,False,1427846400,-0.222521,0.974928,1.000000e+00,0.000000,6.123234e-17,1.0,0.021516,0.999769
2015-04-02,57.628870,58.375509,57.569923,57.687813,3461306,0.0,0.0,-3.318304e+06,5768860,-0.575210,5.440640e+05,100.000000,5.030253,5.030253,-425504.824290,1000.000000,57.799155,0.000000,57.609221,57.766405,57.452036,0.545693,0.750000,0.0,0.0,57.779507,58.781577,56.777436,3.468602,0.454248,0.0,0.0,57.157307,58.375509,57.766408,2.114596,0.435483,0.000000,0.012539,0.002508,...,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,7.692173,43.548280,37.347950,-56.451720,0.000000,57.582683,0.000000,3.846073,0.769215,3.076859,0.273219,0.272846,0.273219,2015,4,14,2,3,92,False,False,False,False,False,False,1427932800,-0.900969,0.433884,9.781476e-01,0.207912,6.123234e-17,1.0,0.004304,0.999991
2015-04-06,57.471678,58.414802,57.412731,58.198669,3101218,0.0,0.0,-1.554863e+06,8870078,-0.175293,6.926660e+05,100.000000,-1.904780,1.562736,36919.899647,1008.855538,57.872429,0.000000,57.805703,58.376068,57.235339,1.973385,0.844487,0.0,0.0,57.855916,58.857987,56.853845,3.464021,0.671023,0.0,0.0,57.157307,58.414802,57.786054,2.175382,0.828125,0.000000,0.062972,0.014601,...,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,30.967687,82.812473,52.502791,-17.187527,0.000000,57.811097,0.000000,5.494728,1.714317,3.780411,0.885554,0.881656,1.161192,2015,4,15,6,0,96,False,False,False,False,False,False,1428278400,1.000000,0.000000,5.000000e-01,0.866025,6.123234e-17,1.0,-0.064508,0.997917
2015-04-07,58.375516,58.375516,57.314499,57.353798,1719055,0.0,0.0,-3.146573e+06,7151023,-0.297151,3.862307e+05,79.315845,-4.243866,-0.372798,2507.390506,994.209957,57.841396,0.000000,57.692727,58.322926,57.062528,2.184674,0.231094,0.0,0.0,57.812255,58.829062,56.795447,3.517619,0.274561,0.0,0.0,57.157307,58.414802,57.786054,2.179642,0.156256,0.000000,0.034370,0.018555,...,0.000000,41.924049,0.000000,0.000000,0.000000,89.182944,23.923445,15.625599,47.328784,-84.374401,0.000000,57.635893,0.000000,2.331071,1.837668,0.493403,-1.451702,-1.462343,-0.307367,2015,4,15,7,1,97,False,False,False,False,False,False,1428364800,0.623490,0.781831,3.090170e-01,0.951057,6.123234e-17,1.0,-0.081676,0.996659
2015-04-08,59.102514,59.986695,58.591655,59.947399,5736940,0.0,0.0,2.267171e+06,12887963,0.

*Defining functions*

**The first one is to ensure all the columns are of the right data type**

In [61]:
def correct_cols(df):

  for col in df.columns[1:80]:
    df[col] = df[col].astype('float')
  
  for col in df.columns[-10:]:
    df[col] = df[col].astype('float')

  for col in df.columns[80:-10]:
    df[col] = df[col].astype('category')

  return df

**Create a Lags function to predict for days into the future**

In [62]:
def create_lags(df, lag_size):

  shiftdays = lag_size
  shift = -shiftdays
  df['Close_lag'] = df['Close'].shift(shift)
  return df, shift

**Create a function to split the data into training and testing sets and then further into X and Y**

In [63]:
def split_data(df, train_pct, shift):

  train_part = int(len(df) * train_pct)

  train = df.iloc[:train_part, :]
  test = df.iloc[train_part: , :]

  X_train = train.iloc[:shift, 1:-1]
  X_test = test.iloc[:shift, 1:-1]

  Y_train = train['Close_lag'][:shift]
  Y_test = test['Close'][:shift]

  return X_train, X_test, Y_train, Y_test, train, test

**Check system for Plotly package, else install it. This is used to create interactive visualizations**

In [64]:
!{sys.executable} -m pip install plotly 
import plotly.graph_objs as go 

**Create a function to plot results comparing actual and predicted results**

In [65]:
def plot_visuals(train, test, pred, ticker, w, h, shift_days, name):

  D1 = go.Scatter(x=train.index, y=train['Close'], name = 'Training Data') 
  D2 = go.Scatter(x=test.index[:shift], y = test['Close'], name = 'Original Testing')
  D3 = go.Scatter(x=test.index[:shift], y=pred,name = 'Predicted Testing')

  line = {'data': [D1,D2,D3],
          'layout': {
              'xaxis' :{'title': 'Date'},
              'yaxis' :{'title': '$tonks'},
              'title' : name + ' - ' + ticker + ' - ' + str(shift_days)
          }} 

  fig = go.Figure(line)

  fig.show()  

**Installing and importing models from Scikit Learn**

In [66]:
!{sys.executable} -m pip install sklearn 
from sklearn.metrics import mean_squared_error 
from sklearn.linear_model import LinearRegression

# Import Artificial Neural Network and Scaler for giving it input
from sklearn.neural_network import MLPRegressor 
from sklearn.preprocessing import StandardScaler

**Create a function for a Linear Regression model**

In [67]:
def lin_reg(X_train, X_test, Y_train, Y_test):

  lr = LinearRegression()
  lr.fit(X_train, Y_train)
  lr_pred = lr.predict(X_test)
  lr_mse = mean_squared_error(Y_test, lr_pred)
  lr_r2 = lr.score(X_test, Y_test)

  print('Linear Regression R2: {}'.format(lr_r2))
  print('Linear Regression MSE: {}'.format(lr_mse))

  return lr_pred

**Create a function for ANN**

In [68]:
def ann_model(X_train, X_test, Y_train, Y_test):

  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train_scaled = scaler.transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  mlp = MLPRegressor(random_state = 1, max_iter = 1000, hidden_layer_sizes = (100,),
                     activation = 'identity', learning_rate = 'adaptive').fit(X_train_scaled, Y_train)
  mlp_pred = mlp.predict(X_test_scaled)
  mlp_mse = mean_squared_error(Y_test, mlp_pred)
  mlp_r2 = mlp.score(X_test_scaled, Y_test)

  print('ANN R2 Score: {}'.format(mlp_r2))
  print('ANN MSE: {}'.format(mlp_mse))

  return mlp_pred

**Create a function to calculate profit/loss using each model**

In [69]:
def calculate_profit(test_df, pred, j):
  pd.set_option('mode.chained_assignment', None)

  test_df['pred'] = np.nan
  test_df['pred'].iloc[:-j] = pred
  test_df['change'] = test_df['Close_lag'] - test_df['Close']
  test_df['change_pred'] = test_df['pred'] - test_df['Close']
  test_df['money_made'] = np.where(test_df['change_pred']/test_df['change'] > 0, 1, -1) 
  test_df['profit'] = np.abs(test['change']) * test_df['money_made']
  profit_dollars = test['profit'].sum()
  print('Money that would have been made: $ ' + str(round(profit_dollars,1)))
  profit_days = len(test_df[test_df['money_made'] == 1])
  print('Percentage of good trading days: ' + str( round(profit_days/(len(test_df)-j),2))     )

  return test_df, profit_dollars

**Now that all the functions have been created, let us loop through the days in shift to get predictions for each shift**

In [70]:
for j in shifts:

  print(str(j) + ' days out:')
  print('------------')
  df_lag, shift = create_lags(df, j)
  df_lag = correct_cols(df_lag)
  X_train, X_test, Y_train, Y_test, train, test = split_data(df, train_pct, shift)


  print("Linear Regression")
  lr_pred = lin_reg(X_train, X_test, Y_train, Y_test)
  test2, profit_dollars = calculate_profit(test, lr_pred, j)
  plot_visuals(train, test, lr_pred, ticker, w, h, j, 'Linear Regression')


  print("ANN")
  mlp_pred = ann_model(X_train, X_test, Y_train, Y_test)
  test2, profit_dollars = calculate_profit(test, mlp_pred, j)
  plot_visuals(train, test, mlp_pred, ticker, w, h, j, 'Artificial Neural Network')
  print('------------')

1 days out:
------------
Linear Regression
Linear Regression R2: 0.99071600151493
Linear Regression MSE: 1.8343627493640813
Money that would have been made: $ 34.8
Percentage of good trading days: 0.52


ANN
ANN R2 Score: 0.9834649887683519
ANN MSE: 3.2670415352209776
Money that would have been made: $ 7.3
Percentage of good trading days: 0.49


------------
5 days out:
------------
Linear Regression
Linear Regression R2: 0.9260463711467841
Linear Regression MSE: 14.486562772457226
Money that would have been made: $ 225.9
Percentage of good trading days: 0.56


ANN
ANN R2 Score: 0.874173748971691
ANN MSE: 24.64774091833211
Money that would have been made: $ 144.8
Percentage of good trading days: 0.53


------------
10 days out:
------------
Linear Regression
Linear Regression R2: 0.7957141707660244
Linear Regression MSE: 40.04269292885422
Money that would have been made: $ 167.9
Percentage of good trading days: 0.58


ANN
ANN R2 Score: 0.7156881108159967
ANN MSE: 55.72884676977901
Money that would have been made: $ 292.8
Percentage of good trading days: 0.55


------------
